In [ ]:
!pip install Morfessor
from google.colab import drive
drive.mount("/content/drive/")
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
print(device)

Mounted at /content/drive/
cuda


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
INDIC_NLP_LIB_HOME = "/content/drive/MyDrive/neural_machine_translation/nmt_iitk/indic_nlp_library"
INDIC_NLP_RESOURCES = "/content/drive/MyDrive/neural_machine_translation/nmt_iitk/indic_nlp_resources"
import sys
sys.path.append(r"{}".format(INDIC_NLP_LIB_HOME))
from indicnlp import common, loader
common.set_resources_path(INDIC_NLP_RESOURCES)
loader.load()

sys.path.append("/content/drive/MyDrive/neural_machine_translation/iter5/code")
from data_preprocessing import retrieve_sents
en_be_src, en_be_trg = retrieve_sents("/content/drive/MyDrive/neural_machine_translation/train_data1.json", "English-Bengali")
en_hi_src, en_hi_trg = retrieve_sents("/content/drive/MyDrive/neural_machine_translation/train_data1.json", "English-Hindi")
en_gu_src, en_gu_trg = retrieve_sents("/content/drive/MyDrive/neural_machine_translation/train_data1.json", "English-Gujarati")
en_ka_src, en_ka_trg = retrieve_sents("/content/drive/MyDrive/neural_machine_translation/train_data1.json", "English-Kannada")
en_ma_src, en_ma_trg = retrieve_sents("/content/drive/MyDrive/neural_machine_translation/train_data1.json", "English-Malayalam")
en_ta_src, en_ta_trg = retrieve_sents("/content/drive/MyDrive/neural_machine_translation/train_data1.json", "English-Tamil")
en_te_src, en_te_trg = retrieve_sents("/content/drive/MyDrive/neural_machine_translation/train_data1.json", "English-Telgu")
from data_preprocessing import english_preprocess
en_be_src = [english_preprocess(sent) for sent in en_be_src]
en_hi_src = [english_preprocess(sent) for sent in en_hi_src]
en_ka_src = [english_preprocess(sent) for sent in en_ka_src]
en_ma_src = [english_preprocess(sent) for sent in en_ma_src]
en_gu_src = [english_preprocess(sent) for sent in en_gu_src]
en_ta_src = [english_preprocess(sent) for sent in en_ta_src]
en_te_src = [english_preprocess(sent) for sent in en_te_src]

from language import Language, make_language
import pickle

with open("/content/drive/MyDrive/neural_machine_translation/iter5/saves/language_instances/en_lang.pkl", "rb") as f:
    en_lang = pickle.load(f)
with open("/content/drive/MyDrive/neural_machine_translation/iter5/saves/language_instances/hi_lang.pkl", "rb") as f:
    hi_lang = pickle.load(f)
with open("/content/drive/MyDrive/neural_machine_translation/iter5/saves/language_instances/be_lang.pkl", "rb") as f:
    be_lang = pickle.load(f)
with open("/content/drive/MyDrive/neural_machine_translation/iter5/saves/language_instances/gu_lang.pkl", "rb") as f:
    gu_lang = pickle.load(f)
with open("/content/drive/MyDrive/neural_machine_translation/iter5/saves/language_instances/ta_lang.pkl", "rb") as f:
    ta_lang = pickle.load(f)
with open("/content/drive/MyDrive/neural_machine_translation/iter5/saves/language_instances/ka_lang.pkl", "rb") as f:
    ka_lang = pickle.load(f)
with open("/content/drive/MyDrive/neural_machine_translation/iter5/saves/language_instances/te_lang.pkl", "rb") as f:
    te_lang = pickle.load(f)
with open("/content/drive/MyDrive/neural_machine_translation/iter5/saves/language_instances/ma_lang.pkl", "rb") as f:
    ma_lang = pickle.load(f)

In [ ]:
MAX_LENGTH = 40

In [ ]:
# class EncoderRNN(nn.Module):
#     def __init__(self, input_size, embed_size, hidden_size, dropout_p=0.1):
#         super(EncoderRNN, self).__init__()
#         self.hidden_size = hidden_size
#         self.embedding = nn.Embedding(input_size, embed_size)
#         self.gru = nn.GRU(embed_size, hidden_size, batch_first=True, bidirectional=True)
#         self.dropout = nn.Dropout(dropout_p)

#     def forward(self, input):
#         # input: (batch_size, seq_len)
#         embedded = self.dropout(self.embedding(input))
#         output, hidden = self.gru(embedded)
#         # hidden = hidden[0:1] + hidden[1:2]
#         hidden = torch.cat((hidden[0:1], hidden[1:2]), dim=2)
#         return output, hidden

# # based on Bahdanau et al. 2015
# class Attention(nn.Module):
#     def __init__(self, hidden_size):
#         super(Attention, self).__init__()
#         self.W = nn.Linear(hidden_size, 2 * hidden_size)
#         self.U = nn.Linear(2 * hidden_size, 2 * hidden_size)
#         self.V = nn.Linear(2 * hidden_size, 1)

#     def forward(self, query, keys):
#         # query: [batch_size, 1, hidden_size]
#         # keys: [batch_size, seq_len, hidden_size]

#         scores = self.V(torch.tanh(self.W(query) + self.U(keys))) # scores: [batch_size, seq_len, 1]
#         scores = scores.squeeze(2).unsqueeze(1) # scores: [batch_size, 1, seq_len]
#         weights = F.softmax(scores, dim=-1)
#         context = torch.bmm(weights, keys) # batch matrix multiplication
#         return context, weights

# class AttentionDecoderRNN(nn.Module):
#     def __init__(self, hidden_size, embed_size, output_size, dropout_p=0.1, SOS_TOKEN=0):
#         super(AttentionDecoderRNN, self).__init__()
#         self.embedding = nn.Embedding(output_size, embed_size)
#         self.attention = Attention(hidden_size)
#         self.gru = nn.GRU(embed_size + 2 * hidden_size, 2 * hidden_size, batch_first=True)  # Adjusted hidden size
#         self.dense = nn.Linear(2 * hidden_size, output_size)  # Adjusted input size
#         self.dropout = nn.Dropout(dropout_p)
#         self.MAX_LENGTH = MAX_LENGTH
#         self.SOS_TOKEN = SOS_TOKEN

#     def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
#         B = encoder_outputs.size(0)
#         decoder_input = torch.empty(B, 1, dtype=torch.long, device=device).fill_(self.SOS_TOKEN)
#         decoder_hidden = encoder_hidden
#         decoder_outputs = []
#         attentions = []

#         for i in range(self.MAX_LENGTH):
#             embed = self.dropout(self.embedding(decoder_input))
#             query = decoder_hidden.permute(1, 0, 2) # query: [1, B, hidden_size]
#             context, attention_wt = self.attention(query, encoder_outputs)
#             output, decoder_hidden = self.gru(torch.cat((embed, context), dim=2), decoder_hidden)
#             output = self.dense(output)

#             decoder_outputs.append(output)
#             attentions.append(attention_wt)

#             if target_tensor is not None:           # teacher_forcing
#                 decoder_input = target_tensor[:, i].unsqueeze(1)
#             else:                                   # autoregressive
#                 _, topi = output.topk(1)
#                 decoder_input = topi.squeeze(1).detach()

#         decoder_outputs = F.log_softmax(torch.cat(decoder_outputs, dim=1), dim=-1)
#         attentions = torch.cat(attentions, dim=1)

#         return decoder_outputs, decoder_hidden, attentions

## MODEL ARCHITECTURE

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, embed_size, hidden_size, dropout_p=0.1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, embed_size)
        self.gru = nn.GRU(embed_size, hidden_size, batch_first=True, bidirectional=True)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input):
        # input: (batch_size, seq_len)
        embedded = self.dropout(self.embedding(input))
        output, hidden = self.gru(embedded)
        hidden = (hidden[0] + hidden[1]).unsqueeze(0)
        # print(hidden.shape)
        return output, hidden

# based on Bahdanau et al. 2015
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.W = nn.Linear(hidden_size, hidden_size)
        # self.U = nn.Linear(hidden_size, hidden_size)
        self.U = nn.Linear(2 * hidden_size, hidden_size)
        self.V = nn.Linear(hidden_size, 1)

    def forward(self, query, keys):
        # query: [batch_size, 1, hidden_size]
        # keys: [batch_size, seq_len, hidden_size]

        scores = self.V(torch.tanh(self.W(query) + self.U(keys))) # scores: [batch_size, seq_len, 1]
        scores = scores.squeeze(2).unsqueeze(1) # scores: [batch_size, 1, seq_len]
        weights = F.softmax(scores, dim=-1)
        context = torch.bmm(weights, keys) # batch matrix multiplication
        return context, weights

class AttentionDecoderRNN(nn.Module):
    def __init__(self, hidden_size, embed_size, output_size, dropout_p=0.1, SOS_TOKEN=1):
        super(AttentionDecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, embed_size)
        self.attention = Attention(hidden_size)
        # self.gru = nn.GRU(embed_size + hidden_size, hidden_size, batch_first=True)
        self.gru = nn.GRU(embed_size + 2 * hidden_size, hidden_size, batch_first=True)
        self.dense = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_p)
        self.SOS_TOKEN = SOS_TOKEN

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        B = encoder_outputs.size(0)
        decoder_input = torch.empty(B, 1, dtype=torch.long, device=device).fill_(self.SOS_TOKEN)
        decoder_hidden = encoder_hidden
        decoder_outputs = []
        attentions = []

        for i in range(MAX_LENGTH):
            embed = self.dropout(self.embedding(decoder_input))
            query = decoder_hidden.permute(1, 0, 2) # query: [1, B, hidden_size]
            context, attention_wt = self.attention(query, encoder_outputs)
            output, decoder_hidden = self.gru(torch.cat((embed, context), dim=2), decoder_hidden)
            output = self.dense(output)

            decoder_outputs.append(output)
            attentions.append(attention_wt)

            if target_tensor is not None:           # teacher_forcing
                decoder_input = target_tensor[:, i].unsqueeze(1)
            else:                                   # autoregressive
                _, topi = output.topk(1)
                decoder_input = topi.squeeze(1).detach()

        decoder_outputs = F.log_softmax(torch.cat(decoder_outputs, dim=1), dim=-1)
        attentions = torch.cat(attentions, dim=1)

        return decoder_outputs, decoder_hidden, attentions

## DATA LOADING

In [ ]:
from dataloading import prepare_indices, TranslationDataset

In [ ]:
en_be_src_i, en_be_trg_i = prepare_indices(en_lang, en_be_src, MAX_LENGTH), prepare_indices(be_lang, en_be_trg, MAX_LENGTH)
en_gu_src_i, en_gu_trg_i = prepare_indices(en_lang, en_gu_src, MAX_LENGTH), prepare_indices(gu_lang, en_gu_trg, MAX_LENGTH)
en_hi_src_i, en_hi_trg_i = prepare_indices(en_lang, en_hi_src, MAX_LENGTH), prepare_indices(hi_lang, en_hi_trg, MAX_LENGTH)
en_ka_src_i, en_ka_trg_i = prepare_indices(en_lang, en_ka_src, MAX_LENGTH), prepare_indices(ka_lang, en_ka_trg, MAX_LENGTH)
en_ma_src_i, en_ma_trg_i = prepare_indices(en_lang, en_ma_src, MAX_LENGTH), prepare_indices(ma_lang, en_ma_trg, MAX_LENGTH)
en_ta_src_i, en_ta_trg_i = prepare_indices(en_lang, en_ta_src, MAX_LENGTH), prepare_indices(ta_lang, en_ta_trg, MAX_LENGTH)
en_te_src_i, en_te_trg_i = prepare_indices(en_lang, en_te_src, MAX_LENGTH), prepare_indices(te_lang, en_te_trg, MAX_LENGTH)

## TRAINING LOOP

In [ ]:
# for time estimation and %progressing
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

def train_epoch(dataloader, encoder, decoder, encoder_optim, decoder_optim, criterion):
    total_loss = 0
    for i, data in enumerate(dataloader):
        if (i + 1) % 400 == 0:
            print(f"Batch: {i + 1} / {len(dataloader)} loaded | Current Loss: {total_loss / (i + 1)}")
        input_tensor, target_tensor = data

        encoder_optim.zero_grad()
        decoder_optim.zero_grad()

        encoder_output, encoder_hidden = encoder(input_tensor)
        decoder_outputs, _, _ = decoder(encoder_output, encoder_hidden, target_tensor)
        loss = criterion(
            decoder_outputs.view(-1, decoder_outputs.size(-1)),
            target_tensor.view(-1)
        )
        loss.backward()

        encoder_optim.step()
        decoder_optim.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)


def train(dataloader, encoder, decoder, n_epochs, encoder_save_path, decoder_save_path, criterion, encoder_optim, decoder_optim):
    start = time.time()
    losses = []

    for epoch in range(n_epochs):
        loss = train_epoch(dataloader, encoder, decoder, encoder_optim, decoder_optim, criterion)
        print(f"Epoch: {epoch + 1} / {n_epochs} | Loss: {loss}")
        print(f"Time Elapsed: {timeSince(start, (epoch + 1) / n_epochs)}")
        losses.append(loss)
        torch.save(encoder.state_dict(), encoder_save_path)
        torch.save(decoder.state_dict(), decoder_save_path)

    return losses

## ENGLISH-BENGALI

In [ ]:
device

device(type='cuda')

In [ ]:
from torch.utils.data import Dataset, DataLoader, RandomSampler

BATCH_SIZE = 16
en_be_dataset = TranslationDataset(en_lang, be_lang, en_be_src_i, en_be_trg_i, device)
en_be_sampler = RandomSampler(en_be_dataset)
en_be_dataloader = DataLoader(en_be_dataset, sampler=en_be_sampler, batch_size=BATCH_SIZE)

In [ ]:
src, trg = next(iter(en_be_dataloader))
src.shape, trg.shape

(torch.Size([16, 40]), torch.Size([16, 40]))

In [ ]:
HIDDEN_SIZE = 256
EMBED_SIZE = 512
INPUT_SIZE = en_lang.n_words
# OUTPUT_SIZE = hi_lang.n_words
OUTPUT_SIZE = be_lang.n_words
N_EPOCHS=10

criterion = nn.NLLLoss(ignore_index=hi_lang.word2idx["<PAD>"])
encoder = EncoderRNN(INPUT_SIZE, EMBED_SIZE, HIDDEN_SIZE).to(device)
decoder = AttentionDecoderRNN(HIDDEN_SIZE, EMBED_SIZE, OUTPUT_SIZE).to(device)
encoder_optim = torch.optim.Adam(encoder.parameters())
decoder_optim = torch.optim.Adam(decoder.parameters())

# losses = train(en_hi_dataloader,
#                encoder,
#                decoder,
#                N_EPOCHS,
#                "/content/drive/MyDrive/neural_machine_translation/iter4/saves/en_hi_lstm_attn_encoder_iter4.pth",
#                "/content/drive/MyDrive/neural_machine_translation/iter4/saves/en_hi_lstm_attn_decoder_iter4.pth",
#                criterion,
#                encoder_optim,
#                decoder_optim
# )

encoder.load_state_dict(torch.load("/content/drive/MyDrive/neural_machine_translation/iter4/saves/en_hi_bigru_attn_encoder_iter4.pth"))

<All keys matched successfully>

In [ ]:
losses = train(en_be_dataloader,
               encoder,
               decoder,
               N_EPOCHS,
               "/content/drive/MyDrive/neural_machine_translation/iter4/saves/en_be_bigru_attn_encoder_iter4.pth",
               "/content/drive/MyDrive/neural_machine_translation/iter4/saves/en_be_bigru_attn_decoder_iter4.pth",
               criterion,
               encoder_optim,
               decoder_optim
)

Batch: 400 / 4304 loaded | Current Loss: 7.569382841587067
Batch: 800 / 4304 loaded | Current Loss: 7.207891881465912
Batch: 1200 / 4304 loaded | Current Loss: 6.976794905265172
Batch: 1600 / 4304 loaded | Current Loss: 6.8058427268266675
Batch: 2000 / 4304 loaded | Current Loss: 6.669567894697189
Batch: 2400 / 4304 loaded | Current Loss: 6.556768497029941
Batch: 2800 / 4304 loaded | Current Loss: 6.464894703456333
Batch: 3200 / 4304 loaded | Current Loss: 6.385794554948807
Batch: 3600 / 4304 loaded | Current Loss: 6.313049903313319
Batch: 4000 / 4304 loaded | Current Loss: 6.249852312803268
Epoch: 1 / 10 | Loss: 6.20983701822483
Time Elapsed: 23m 55s (- 215m 22s)
Batch: 400 / 4304 loaded | Current Loss: 4.791704937219619
Batch: 800 / 4304 loaded | Current Loss: 4.789106113612652
Batch: 1200 / 4304 loaded | Current Loss: 4.8008152012030285
Batch: 1600 / 4304 loaded | Current Loss: 4.794870827198029
Batch: 2000 / 4304 loaded | Current Loss: 4.803681671380997
Batch: 2400 / 4304 loaded | 

## EVALUATION

In [ ]:
import json
with open("/content/drive/MyDrive/neural_machine_translation/val_data1.json", "rb") as file:
    val_data = json.load(file)
en_be_eval = []
en_be_id = []
en_gu_eval = []
en_gu_id = []
en_hi_eval = []
en_hi_id = []
en_ka_eval = []
en_ka_id = []
en_ma_eval = []
en_ma_id = []
en_ta_eval = []
en_ta_id = []
en_te_eval = []
en_te_id = []

for lang_pair, data in val_data.items():
    for _, d_entry in data.items():
        for id, content in d_entry.items():
            if lang_pair == "English-Bengali":
                en_be_eval.append(content["source"])
                en_be_id.append(id)
            elif lang_pair == "English-Gujarati":
                en_gu_eval.append(content["source"])
                en_gu_id.append(id)
            elif lang_pair == "English-Hindi":
                en_hi_eval.append(content["source"])
                en_hi_id.append(id)
            elif lang_pair == "English-Kannada":
                en_ka_eval.append(content["source"])
                en_ka_id.append(id)
            elif lang_pair == "English-Malayalam":
                en_ma_eval.append(content["source"])
                en_ma_id.append(id)
            elif lang_pair == "English-Tamil":
                en_ta_eval.append(content["source"])
                en_ta_id.append(id)
            elif lang_pair == "English-Telgu":
                en_te_eval.append(content["source"])
                en_te_id.append(id)

# en_be_src, en_be_trg = retrieve_sents("/content/drive/MyDrive/neural_machine_translation/val_data1.json", "English-Bengali")
# en_hi_src, en_hi_trg = retrieve_sents("/content/drive/MyDrive/neural_machine_translation/val_data1.json", "English-Hindi")
# en_gu_src, en_gu_trg = retrieve_sents("/content/drive/MyDrive/neural_machine_translation/val_data1.json", "English-Gujarati")
# en_ka_src, en_ka_trg = retrieve_sents("/content/drive/MyDrive/neural_machine_translation/val_data1.json", "English-Kannada")
# en_ma_src, en_ma_trg = retrieve_sents("/content/drive/MyDrive/neural_machine_translation/val_data1.json", "English-Malayalam")
# en_ta_src, en_ta_trg = retrieve_sents("/content/drive/MyDrive/neural_machine_translation/val_data1.json", "English-Tamil")
# en_te_src, en_te_trg = retrieve_sents("/content/drive/MyDrive/neural_machine_translation/val_data1.json", "English-Telgu")

en_be_src = [english_preprocess(sent) for sent in en_be_src]
en_hi_src = [english_preprocess(sent) for sent in en_hi_src]
en_ka_src = [english_preprocess(sent) for sent in en_ka_src]
en_ma_src = [english_preprocess(sent) for sent in en_ma_src]
en_gu_src = [english_preprocess(sent) for sent in en_gu_src]
en_ta_src = [english_preprocess(sent) for sent in en_ta_src]
en_te_src = [english_preprocess(sent) for sent in en_te_src]

In [ ]:
# load the model
encoder.load_state_dict(torch.load("/content/drive/MyDrive/neural_machine_translation/iter4/saves/en_be_bigru_attn_encoder_iter4.pth"))
decoder.load_state_dict(torch.load("/content/drive/MyDrive/neural_machine_translation/iter4/saves/en_be_bigru_attn_decoder_iter4.pth"))

In [ ]:
def tensorFromSentence(lang, sentence):
    indexes = lang.sentence_to_indices(sentence)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(1, -1)

def evaluate(encoder, decoder, src_sents, src_lang, trg_lang, EOS_TOKEN=1):
  with torch.no_grad():

    trg_sents = []

    for i, src_sent in enumerate(src_sents):
      if i % 1000 == 0:
        print(f"Number of sentences translated: {i}")
      input_tensor = tensorFromSentence(src_lang, src_sent)
      encoder_outputs, encoder_hidden = encoder(input_tensor)
      decoder_outputs, decoder_hidden, decoder_attn = decoder(encoder_outputs, encoder_hidden)
      _, topi = decoder_outputs.topk(1)
      decoded_ids = topi.squeeze()

      decoded_words = []

      for idx in decoded_ids:
        if idx.item() == EOS_TOKEN:
          break
        decoded_words.append(trg_lang.idx2word[idx.item()])

      trg_sents.append(" ".join(decoded_words))

    return trg_sents

In [ ]:
en_be_trans = evaluate(encoder, decoder, en_be_eval, en_lang, be_lang)

In [ ]:
len(en_be_eval), len(en_be_trans)

In [ ]:
import pandas as pd
df = pd.DataFrame({'ID': en_be_id, 'Translation': en_be_trans})
csv_path = "/content/drive/MyDrive/neural_machine_translation/iter4/bengali.csv"
df.to_csv(csv_path, index=False)